# Tutorial 4: Using Schemas in RadPrompter

In this tutorial, we'll explore how to use **Schemas** in RadPrompter to extract more structured data from radiology reports. Schemas allow us to define the specific data elements we want to extract and provide hints and constraints to guide the model's output.

## Prompt

As always, we start by importing the `Prompt` class and creating a prompt object from a TOML file:

In [1]:
from radprompter import Prompt

prompt = Prompt('04_Using-Schemas.toml')
prompt

The key feature of this TOML file is the `[SCHEMAS]` section. Here's an example schema:

```toml
[SCHEMAS.PulmonaryEmbolism]
variable_name = "Pulmonary Embolism"
intro_prompt = """
Carefully review the provided chest CT report (in the <report> tag). Ensure that each data element is accurately captured.
Here is the report:
<report>
{{report}}
</report>
Please pay attention to the following details:
- Your attention to detail is crucial for maintaining the integrity of the medical records.
- You should not confabulate information, and if something is not mentioned, you should assume that it is `Absent` unless otherwise stated.
- The report may contain additional information that is not relevant to the requested data elements. Please ignore that information.
- We are interested at findings at the time of scan, not the previous ones, so only consider the impression and findings sections of the report.
- Do not print anything else other than the provided output format.
"""
type = "select"
options = ["Present", "Absent"]
show_options_in_hint = true
hint = """
Indicate `Present` if the report explicitly mentions the patient has pulmonary embolism in their CT scan.
Indicate `Absent` if pulmonary embolism is not seen or if a previously observed pulmonary embolism is mentioned as resolved.
"""
```

The bare minimum for a schema is the `variable_name`, which defines the name of the data element to extract. 

If the `type` is set to "select", you can provide a list of `options` for the model to choose from. Setting `show_options_in_hint` to `true` will include these options in the hint text.

Attributes of each schema will replace the `{{}}` placeholders in the original prompt. Additionally, they schemas can also contain `{{}}` placeholders themselves.

Let's look at the schemas defined in our prompt:


In [2]:
prompt.schemas[0]

The first schema contains the introduction prompt with the radiology report and general instructions. You see that the `{{intro_prompt}}` placeholder is now replaced with the schema's `intro_prompt` attribute. Also you can see that the replaced prompt contains a `{{report}}` placeholder that will be populated when we pass in reports.

Subsequent schemas focus on specific data elements:

In [3]:
prompt.schemas[1]

## Client & Engine

We'll use the `vLLMClient` and `RadPrompter` engine as in previous tutorials:

In [4]:
from radprompter import RadPrompter, vLLMClient

client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    temperature = 0.0,
    seed=42
)

engine = RadPrompter(
    client=client,
    prompt=prompt, 
    output_file="output_tutorial_4.csv",
    concurrency=2,
    hide_blocks=False,
)

The `hide_blocks` parameter is an important setting that determines how the model processes the schemas.

When `hide_blocks=False`, the model will see all of the previously processed schemas when working on the current schema. This means that when the model is answering questions for a particular schema, it has access to the information it has already extracted from the previous schemas.

On the other hand, when `hide_blocks=True`, the schemas will be processed independently. The model will only see each schema in isolation, without having access to its answers to the previous schemas.

The recommendation is to use `hide_blocks=False` when the schemas are related to each other, as in this case where all the schemas are extracting information about pulmonary embolism. The model's answers to earlier questions about the presence and location of the embolism are relevant to answering subsequent questions.

However, you should use `hide_blocks=True` when the pathologies or the extracted information are independent. For example, if you want to extract five different, unrelated pathologies from a single report, you would set `hide_blocks=True` so that the model's answers about one pathology don't influence its answers about the others.

And we run it on our sample reports:

In [5]:
import glob

report_files = glob.glob("../../sample_reports/*.txt")

reports = []
for file in report_files:
    with open(file, "r") as f:
        reports.append({"report": f.read(), "file_name": file})

engine(reports)

Processing items: 100%|██████████| 3/3 [00:02<00:00,  1.20it/s]


The engine will process each report using **ALL** the schemas in the prompt and save the results to `output_tutorial_4.csv`.

In [6]:
import pandas as pd

df = pd.read_csv("output_tutorial_4.csv", index_col='index')
df

,Pulmonary Embolism_response,Left_response,Right_response,Acute_response,Chronic_response,RightHeartStrain_response,PulmonaryArteryHypertension_response,report,file_name
index,,,,,,,,,
1,Present,Yes,Yes,No,No,No,No,Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt
0,Present,Yes,Yes,Yes,No,Yes,No,Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
2,Present,Yes,Yes,No,Yes,No,Yes,Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt


As you can see, response to each schema are recorded in `{variable_name}_response` column. For example the response for the first schema is stored in `Pulmonary Embolism_response`. Note that you can employ multi-turn prompting for processing schemas. In that case, the model responses would be recorded in `{variable_name}_response_1` and `{variable_name}_response_2` columns.

Finally, we save the log:

In [7]:
engine.save_log("log_tutorial_4.log")

with open("log_tutorial_4.log", "r") as f:
    print(f.read())

RadPrompter Version: 1.0.8
Model: meta-llama/Meta-Llama-3-8B-Instruct
Prompt TOML: /Users/bardiakhosravi/Desktop/GitHub/RadPrompter/tutorials/04_Using-Schemas/04_Using-Schemas.toml
Prompt Version: 0.1
Prompt Hash: 2254abf376cfa05a5026fdcc6cf6e428
Concurrency Factor: 2
Start Time: 2024-05-19 15:37:06
End Time: 2024-05-19 15:37:09
Duration: 3.0
Number of Items: 3
Average Processing Time: 1.0


-------------------- *** - Prompt Content - *** --------------------

[METADATA]

version = 0.1
description = "A sample prompt for RadPrompter with no CoT"

[PROMPTS]

system_prompt = "You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements."

user_prompt_intro = "{{intro_prompt}}\n"

user_prompt_no_cot = """
I want you to extract the following data element from the report: 
{{hint}}

Provide a single answer:

"""

[CONSTRUCTOR]
system = "rdp(system_prompt)"
user = [
"rdp(user_prompt_intro + user_prompt_no_cot)"
]
stop_tags = [
" "
]


[

Schemas are a powerful feature in RadPrompter that allow us to extract structured data from unstructured radiology reports. By defining the data elements we're interested in and providing hints and constraints, we can guide the model to produce the desired output format.